In [ ]:
#import all libraries and dependencies

%matplotlib inline

from time import perf_counter 
import os
import sys
import matplotlib.pyplot as plt
import osmnx as ox 
from collections import Counter
import pygal
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
import matplotlib

#Interactive visualization
import folium
from folium import plugins
# Adds tool to the top right
from folium.plugins import MeasureControl
import branca.colormap as cm

import seaborn as sns; sns.set()

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_moons
from sklearn import metrics

from pygeocoder import Geocoder 

import reverse_geocoder as rg #info. on output:(name: cityname, admin1:state, admin2:county, cc:country code): https://github.com/thampiman/reverse-geocoder
import pprint 

from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
from plotly import tools
import matplotlib.cm as cm
import plotly.graph_objs as go

import random

from kneed.data_generator import DataGenerator
from kneed.knee_locator import KneeLocator

In [ ]:
##  # To Compute Haversine distance
def sphere_dist(o_lat, o_lon, d_lat, d_lon):
    """
    Return distance along great radius between origin and destination coordinates.
    """
    #Define earth radius (km)
    R_earth = 6371
    #Convert degrees to radians
    o_lat, o_lon, d_lat, d_lon = map(np.radians,
                                                             [o_lat, o_lon, 
                                                              d_lat, d_lon])
    #Compute distances along lat, lon dimensions
    dlat = d_lat - o_lat
    dlon = d_lon - o_lon
    
    #Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(o_lat) * np.cos(d_lat) * np.sin(dlon/2.0)**2
    return 2 * R_earth * np.arcsin(np.sqrt(a))

def radian_conv(degree):
    """
    Return radian.
    """
    return  np.radians(degree) 

In [ ]:
#Courtesy of https://stackoverflow.com/questions/26079881/kl-divergence-of-two-gmms. Here the difference is that we take the squared root, so it's a proper metric

def gmm_js(gmm_p, gmm_q, n_samples=10**5):
    X = gmm_p.sample(n_samples)[0]
    log_p_X = gmm_p.score_samples(X)
    log_q_X = gmm_q.score_samples(X)
    log_mix_X = np.logaddexp(log_p_X, log_q_X)

    Y = gmm_q.sample(n_samples)[0]
    log_p_Y = gmm_p.score_samples(Y)
    log_q_Y = gmm_q.score_samples(Y)
    log_mix_Y = np.logaddexp(log_p_Y, log_q_Y)
    
    return np.sqrt((log_p_X.mean() - (log_mix_X.mean() - np.log(2))
            + log_q_Y.mean() - (log_mix_Y.mean() - np.log(2))) / 2)

In [ ]:
# Input Training Dataset for the Model

def ReadFromCSV(FilePath, sckipLines = 0, sep = ','):  

    #Read and clean the csv file in FilePath ignoring the first sckipLines lines.

    #(omits non-ascii characters from columns' name)

    xa = pd.read_csv(FilePath, header=sckipLines, sep = sep, low_memory=False)

    return xa



In [ ]:
def data_manipulation(J):
    
    ## clean the data from non meaningful lat. and long.
    # ##### Valid range of latitude and longitude for India: lat range( 8°4′ N to 37°6′ N) & long. range( 68°7′ E to 97°25′ E)
    
    J=J[(J['origin_lat'] >= 8) & (J['origin_lat'] <= 38)  & (J['destination_lat'] >= 8) & (J['destination_lat']<=38)]
    J=J[(J['origin_lng'] >= 67) & (J['origin_lng'] <= 98)  & (J['destination_lng'] >= 67) & (J['destination_lng']<=98)]                            
               
    J_clean=J.loc[(J['origin_lng'] >= 67) & (J['destination_lng'] <= 98)]
    
    
    # ### Jitter in lat/long of cols ==> The second decimal place is worth up to 1.1 km

    J_clean['origin_lat']=J_clean['origin_lat'].map(lambda x: x +np.random.rand()/5.0)
    J_clean['origin_lng']=J_clean['origin_lng'].map(lambda x: x +np.random.rand()/5.0)

    J_clean['destination_lat']=J_clean['destination_lat'].map(lambda x: x +np.random.rand()/5.0)
    J_clean['destination_lng']=J_clean['destination_lng'].map(lambda x: x +np.random.rand()/5.0)
    
    J_clean=J_clean.dropna()
    J_clean['distance'] = sphere_dist(J_clean['origin_lat'], J_clean['origin_lng'], 
                                   J_clean['destination_lat'] , J_clean['destination_lng'])
    J_clean["datetime"] = pd.to_datetime(J_clean["datetime"]) #for time series needed to be dtype: datetime64[ns, UTC]
    
    return J_clean



In [ ]:
### Long trip shipment or short trip shipment distances

def vis_route_length_dist(d):

    route_lengths=d['distance']

    long_routes = len([k for k in route_lengths if k > 400]) / len(route_lengths)
    medium_routes = len([k for k in route_lengths if k < 400 and k > 300]) / len(route_lengths)
    short_routes = len([k for k in route_lengths if k < 300]) / len(route_lengths)


    chart = pygal.HorizontalBar()
    chart.title = 'Long, medium, and short routes'
    chart.add('Long', long_routes * 100)
    chart.add('Medium', medium_routes * 100)
    chart.add('Short', short_routes * 100)
    
    return chart
#vis_route_length_dist(df)

In [ ]:
 #####. Function of find optimal number of clusters (GMM components)? Based on AIC


def optimal_cluster_k (X):
    
    if X.shape[0]==1 or X.shape[0]==2 :
        n_clusters=1
    
    elif 2< X.shape[0] <= 10:
        n_clusters=2
        
        gmm = GaussianMixture(n_components=2, covariance_type='full',random_state=42).fit(X)
        cluster_labels = gmm.predict(X)
        
        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters

        silhouette_avg = silhouette_score(X, cluster_labels)

        print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)    
        
    elif  11 <= X.shape[0] <= 100:
        
        k_clusters= np.arange(2, X.shape[0]-1,5)
        models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(X) for n in k_clusters]
        kn = KneeLocator(k_clusters, [m.aic(X) for m in models], curve='convex', direction='decreasing')
        knee_k = kn.knee
        
        #print (knee_k)
        
        bic_min = min([m.bic(X) for m in models])
        k_bic=k_clusters[[m.bic(X) for m in models].index(bic_min)]
        
        #print (k_bic)
        
#         plt.plot(k_clusters, [m.bic(X) for m in models], label='BIC') 
#         plt.plot(k_clusters, [m.aic(X) for m in models], label='AIC') 
#         plt.vlines(x=knee_k, ymin=min([m.bic(X)-500 for m in models]), ymax=max([m.bic(X) for m in models]), linestyle='--')
#         plt.legend(loc='best')
#         plt.xlabel('n_components', fontsize=15);

        #start_r=np.min(k_bic,knee_k)
        
        range_n_clusters = np.arange(2, X.shape[0]-1,1)

        for n_clusters in range_n_clusters:

            # Initialize the clusterer with n_clusters value and a random generator
            # seed of 10 for reproducibility.
            #clusterer = KMeans(n_clusters=n_clusters, random_state=10)
            #cluster_labels = clusterer.fit_predict(X)
            gmm = GaussianMixture(n_components=n_clusters, covariance_type='full',random_state=42).fit(X)
            cluster_labels = gmm.predict(X)
            print (cluster_labels)
            # The silhouette_score gives the average value for all the samples.
            # This gives a perspective into the density and separation of the formed
            # clusters
            
            silhouette_avg = silhouette_score(X, cluster_labels)
     
            if silhouette_avg > 0.5:
                print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)    
                break
    elif X.shape[0] > 100:
        
        k_clusters= np.arange(2, 90,10)
        models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(X) for n in k_clusters]
        kn = KneeLocator(k_clusters, [m.aic(X) for m in models], curve='convex', direction='decreasing')
        knee_k = kn.knee
        print (knee_k)
        
        bic_min = min([m.bic(X) for m in models])
        k_bic=k_clusters[[m.bic(X) for m in models].index(bic_min)]
       
        print (k_bic)
        
#         plt.plot(k_clusters, [m.bic(X) for m in models], label='BIC') 
#         plt.plot(k_clusters, [m.aic(X) for m in models], label='AIC') 
#         plt.vlines(x=knee_k, ymin=min([m.bic(X)-500 for m in models]), ymax=max([m.bic(X) for m in models]), linestyle='--')
#         plt.legend(loc='best')
#         plt.xlabel('n_components', fontsize=15);
        
        range_n_clusters = np.arange(knee_k, 90,2) #step was 2 before

        for n_clusters in range_n_clusters:

            # Initialize the clusterer with n_clusters value and a random generator
            # seed of 10 for reproducibility.
            #clusterer = KMeans(n_clusters=n_clusters, random_state=10)
            #cluster_labels = clusterer.fit_predict(X)
            gmm = GaussianMixture(n_components=n_clusters, covariance_type='full',random_state=42).fit(X)
            cluster_labels = gmm.predict(X)

            # The silhouette_score gives the average value for all the samples.
            # This gives a perspective into the density and separation of the formed
            # clusters
            silhouette_avg = silhouette_score(X, cluster_labels)


            if silhouette_avg > 0.5:
                print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)    
                break
    return n_clusters #k_bic if None else knee_k #np.min(n_clusters, X.shape[0]) 
    
#coords = df_train_dict['Telangana_Uttarakhand'].as_matrix(columns=['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng'])

#optim_n= optimal_cluster_k(coords)
#df_train_dict['Telangana_Uttarakhand'].shape[0] #25 

In [ ]:
def GMM_Model(df_train,outputpath)

    #create new column to store origin state_destination state combination
    df_train['shipments_states']=df_train.apply(lambda x:'%s_%s' % (x['origin_state'],x['destination_state']),axis=1) 

    #make a list from all the route in each state_state shipments
    routes_states = df_train['shipments_states'].unique().tolist()

    #create a dictionary to store all dataframes that each stores the single origin state_destination state
    df_train_dict = {state_state: df_train.loc[df_train['shipments_states'] == state_state] for state_state in routes_states}


    k_list=[]
    gmm_tr={}

    for k, v in df_train_dict.items():

        if df_train_dict[k].shape[0]==1:
            print (k)
            print ('This route is single route and one cluster')
        else:
            print (k)
            coords_tr = df_train_dict[k].as_matrix(columns=['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng'])

            optim_tr= optimal_cluster_k(coords_tr)


            gmm_tr[k]= GaussianMixture(n_components=optim_tr, covariance_type='full',random_state=42).fit(coords_tr)

            cluster_labels_tr = gmm_tr[k].predict(coords_tr)      
            probs_tr = gmm_tr[k].predict_proba(coords_tr)                  

            df_train_dict[k]['route_zones']= pd.Series(cluster_labels_tr, index=df_train_dict[k].index)
            df_train_dict[k]['probs_route_zone'] = pd.Series(probs_tr.max(1), index=df_train_dict[k].index)
            df_train_dict[k].to_csv(outputpath + k + '.csv')


    print ('k_list:',k_list)
    
    return 

    
t1_start = perf_counter()
t1_stop = perf_counter() 
  
print("Elapsed time:", t1_stop, t1_start)  
print("Elapsed time during the whole program in minutes:",(t1_stop-t1_start)/60)



In [ ]:
df_all=ReadFromCSV('/Users/mojganmazouchi/Desktop/golorry_route/all_data.csv') 
df=data_manipulation(df_all) #(107498, 11)
#df.shape

GMM_Model(df,/Results2)

In [ ]:
#### For visualization 

### get subset of state to state route which you want to visualize after building the model

def state_state_clusters(ss, data_dict):

    #ss='West Bengal_Telangana' ## input the origin state to destination state with this example format 'Telangana_West Bengal'
                                                              
    # # mark each origin as a point
    df_plot=df_dict[ssr] ## 22 zones detected

    all_routes = df_plot['route_zones'].unique().tolist()

    df_dict_zones = {route_z: df_plot.loc[df_plot['route_zones'] == route_z] for route_z in all_routes}


    m = folium.Map([16.7, 81.095], zoom_start=11,control_scale = True)


    #create a color map
    #color_var = 'route_zones' #what variable will determine the color
    cmap = cm.LinearColormap(['blue', 'red','green', 'orange','pink','black', 'beige', 'white','lightblue','purple','gray','cadetblue'],
                         vmin=df_plot['route_zones'].quantile(0.05), vmax=df_plot['route_zones'].quantile(0.95),
                         caption = 'route_zones')

    #Add the color map legend to your map
    m.add_child(cmap)
    #Add the measurement control bar
    m.add_child(MeasureControl())

  
        ####### Source info. on Folium https://www.kaggle.com/rachan/how-to-folium-for-maps-heatmaps-time-analysis
    for k, v in df_dict_zones.items():


        for index, row in df_dict_zones[k].iterrows():#folium.Marker([lat, lon], popup=str(name)+': '+color+'-'+str(clname), icon=folium.Icon(color=color)).add_to(feature_group)

            folium.Marker([row['origin_lat'], row['origin_lng']],
                                radius=4,fill=True, # Set fill to True
                                fill_color="green",fill_opacity=0.7).add_to(m)

        for index, row in df_dict_zones[k].iterrows():

            folium.Marker([row['destination_lat'], row['destination_lng']],
                                radius=7,
                                fill=True, # Set fill to True
                                color = 'red',
                                fill_opacity=0.7).add_to(m)

            folium.PolyLine([[row['origin_lat'], row['origin_lng']], [row['destination_lat'], row['destination_lng']]],color=cmap(row['route_zones']),line_weight=5).add_to(m)
    
    return m


In [ ]:
##### Check the robustness of a model with shuffle data for specific state_state route

def Robustness_check(ss, df_p,df_q):

df_tr=df

df_check_p=df_tr.copy()
df_check_q=df_tr.sample(n=55000)

shipment_route='West Bengal_Telangana' ## input the origin state to destination state with this example format 'Telangana_West Bengal'
  
df_check_p['shipments_states']=df_check_p.apply(lambda x:'%s_%s' % (x['origin_state'],x['destination_state']),axis=1)
#routes_states = df_check_p['shipments_states'].unique().tolist()
routes_states = [shipment_route]#'Andhra Pradesh_Andhra Pradesh'

df_dict_p = {state_state: df_check_p.loc[df_check_p['shipments_states'] == state_state] for state_state in routes_states}


df_check_q['shipments_states']=df_check_q.apply(lambda x:'%s_%s' % (x['origin_state'],x['destination_state']),axis=1)
df_dict_q = {state_state: df_check_q.loc[df_check_q['shipments_states'] == state_state] for state_state in routes_states}


p_list=[]
q_list=[]
k_list=[]
JSD=[]
gmm_p={}
gmm_q={}
js_dist={}
#for k, v in df_dict.items():
for (k,v), (k2,v2) in zip(df_dict_p.items(), df_dict_q.items()):
    
    if df_dict_p[k].shape[0]==1 & df_dict_q[k2].shape[0]==1:
        
        print ('This route is single route and one cluster')
    else:
        
        coords_p = df_dict_p[k].as_matrix(columns=['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng'])
        coords_q = df_dict_q[k2].as_matrix(columns=['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng'])
        
        optim_p= optimal_cluster_k(coords_p)
        optim_q= optimal_cluster_k(coords_q)
        
     
        gmm_p[k]= GaussianMixture(n_components=optim_p, covariance_type='full',random_state=42).fit(coords_p)
        gmm_q[k2]= GaussianMixture(n_components=optim_q, covariance_type='full',random_state=42).fit(coords_q)
        
        cluster_labels_p = gmm_p[k].predict(coords_p)
        cluster_labels_q = gmm_p[k2].predict(coords_q)
        
        probs_p = gmm_p[k].predict_proba(coords_p)
        probs_q = gmm_q[k2].predict_proba(coords_q) 
        
        js_dist[k]=gmm_js(gmm_p[k], gmm_q[k2], n_samples=100)
        
        if optim_p!= optim_q:

            p_list.append(optim_p)
            q_list.append(optim_q)
            k_list.append(k)
            JSD.append(js_dist[k])            

        #print ('js',js_dist)
        df_dict_p[k]['route_zones']= pd.Series(cluster_labels_p, index=df_dict_p[k].index)
        df_dict_p[k]['probs_route_zone'] = pd.Series(probs_p.max(1), index=df_dict_p[k].index)
        #df_dict[k].to_csv(k + '.csv')
         
print ('p_list:', p_list)
print ('q_list:', q_list)
print ('k_list:',k_list)
print ('JS distance:', js_dist[k])


                